In [1]:
%run headers.ipynb

#### Images   ->  Ultrasound  || Masked  ->  Prediction

In [2]:
# data=[(images_benign,masked_benign),(images_malignant,masked_malignant),(images_normal,masked_normal)]
data=[([],[]),([],[]),([],[])]
data_url=[([],[]),([],[]),([],[])]

count_temp=0

directory="D:/tumor/resized/"
subdir=["benign/","malignant/","normal/"]

#### Load Data:

In [3]:
def LoadData(imgPath,subd,ix,images,masked,images_url,masked_url,check):
    global count_temp
    count_temp+=1
    
    if(count_temp>3):
        for i in range(3):
            data[i][0].clear()
            data[i][1].clear()
        count_temp=1
        
    imgPath=imgPath+subd
    imgNames = os.listdir(imgPath)
    
    for item in imgNames:
        img = plt.imread(imgPath + item)
        if item.endswith(')_mask resized_greyscale.jpg'):
            masked.append(img)
        elif item.endswith(') resized.jpg'):
            images.append(img)
        elif item.endswith(' resized_greyscale.jpg'):
            masked[-1]=np.add(masked[-1], img)

In [4]:
def LoadData_Info_with_print():
    
    print("Information about the dataset : \n")
    
    for i in range(3):
        LoadData(directory,subdir[i],i,data[i][0],data[i][1],data_url[i][0],data_url[i][1],1)

    print(f"\nTotal images : {len(data[0][0])+len(data[1][0])+len(data[2][0])}")
    for i in range(3):
        print(len(data[i][0]))
    print()
    
    print(f"Total masked : {len(data[0][1])+len(data[1][1])+len(data[2][1])}")
    for i in range(3):
        print(len(data[i][1]))
    print()

In [5]:
def LoadData_Info():
    for i in range(3):
        LoadData(directory,subdir[i],i,data[i][0],data[i][1],data_url[i][0],data_url[i][1],0)

In [6]:
def UNet_dataset_with_valid(train_ratio,test_ratio,valid_ratio): # 0.7,0.15,0.15
    
    LoadData_Info()

    l=[]

    test_ratio  = 1 - train_ratio
    valid_ratio = valid_ratio/test_ratio # 0.15/0.3=0.5
    test_ratio  = 1 - valid_ratio
    
    for i in range(3):
        X_train,X_rem,y_train,y_rem=train_test_split(data[i][0], data[i][1], train_size=train_ratio, random_state=42)
        X_valid, X_test, y_valid, y_test=train_test_split(X_rem,y_rem, test_size=test_ratio,random_state=42)
        l.append([X_train, y_train, X_valid, y_valid, X_test, y_test])

        for i in range(6):
            l[-1][i]=np.array(l[-1][i],dtype=object)

    X_train  = []
    y_train  = []
    
    X_valid  = []
    y_valid  = []
    
    X_test   = []
    y_test   = []
    
    for i in range(3):
        X_train.append(l[i][0])
        y_train.append(l[i][1])
        
        X_valid.append(l[i][2])
        y_valid.append(l[i][3])
        
        X_test.append(l[i][4])
        y_test.append(l[i][5])

    
    X_train=np.concatenate(X_train, axis=0)
    X_test=np.concatenate(X_test, axis=0)
    X_valid=np.concatenate(X_valid, axis=0)

    y_train=np.concatenate(y_train, axis=0)
    y_test=np.concatenate(y_test, axis=0)
    y_valid=np.concatenate(y_valid, axis=0)
        
    X_train,y_train=shuffle(X_train, y_train, random_state=0)
    X_test,y_test=shuffle(X_test, y_test, random_state=0)
    X_valid,y_valid=shuffle(X_valid, y_valid, random_state=0)   
    
    X_train=X_train.astype(np.float32)
    y_train=y_train.astype(np.float32)

    X_test=X_test.astype(np.float32)
    y_test=y_test.astype(np.float32)

    X_valid=X_valid.astype(np.float32)
    y_valid=y_valid.astype(np.float32)

    for i in range(len(X_train)):
        X_train[i]=X_train[i].astype(np.float32)
        y_train[i]=y_train[i].astype(np.float32)

    for i in range(len(X_test)):
        X_test[i]=X_test[i].astype(np.float32)
        y_test[i]=y_test[i].astype(np.float32)

    for i in range(len(X_valid)):
        X_valid[i]=X_valid[i].astype(np.float32)
        y_valid[i]=y_valid[i].astype(np.float32)
    
    return [X_train, X_valid, X_test, y_train, y_valid, y_test]

In [7]:
def UNet_dataset(train_ratio): # 0.7,0.3
    
    LoadData_Info()
    
    l=[]
    
    for i in range(3):
        X_train,X_test,y_train,y_test=train_test_split(data[i][0], data[i][1], train_size=train_ratio, random_state=42)
        l.append([X_train, y_train, X_test, y_test])

        for i in range(4):
            l[-1][i]=np.array(l[-1][i],dtype=object)

    X_train = []
    y_train  = []
    X_test = []
    y_test = []
    
    for i in range(3):
        X_train.append(l[i][0])
        y_train.append(l[i][1])
        X_test.append(l[i][2])
        y_test.append(l[i][3])

    X_train=np.concatenate(X_train, axis=0)
    X_test=np.concatenate(X_test, axis=0)
    y_train=np.concatenate(y_train, axis=0)
    y_test=np.concatenate(y_test, axis=0)
    
    X_train,y_train=shuffle(X_train, y_train, random_state=0)
    X_test,y_test=shuffle(X_test, y_test, random_state=0)
    
    X_train=X_train.astype(np.float32)
    y_train=y_train.astype(np.float32)
    X_test=X_test.astype(np.float32)
    y_test=y_test.astype(np.float32)

    for i in range(len(X_train)):
        X_train[i]=X_train[i].astype(np.float32)
        y_train[i]=y_train[i].astype(np.float32)

    for i in range(len(X_test)):
        X_test[i]=X_test[i].astype(np.float32)
        y_test[i]=y_test[i].astype(np.float32)
    
    return [X_train, X_test, y_train, y_test]